In [18]:
from lsst.daf.butler import Butler

import numpy as np

In [19]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

In [20]:
tract = 3828
patch = 3
dataId = {'tract': tract, 'patch': patch}

In [21]:
forced_src = butler.get('forcedSourceTable', dataId=dataId, parameters={'columns':['objectId', 'coord_ra', 'coord_dec','psfFlux', 'psfFluxErr', 'ccdVisitId', 'band']})
ccdvisit = butler.get('ccdVisitTable', dataId=dataId, parameters={'columns':['ccdVisitId', 'expMidptMJD']})
objtable = butler.get('objectTable', dataId=dataId, parameters={'columns':['objectId', 'coord_ra', 'coord_dec', 'g_psfFlux']})

In [16]:
inds = np.unique(forced_src['objectId'])[:5]

In [17]:
inds

array([1649927148640665601, 1649927148640665602, 1649927148640665603,
       1649927148640665604, 1649927148640665605])

In [9]:
fsrc = forced_src.join(ccdvisit, on='ccdVisitId', lsuffix='_caller', rsuffix='_other')[:100] #.loc(inds[0])

Problem is that it's keyed on forcedSourceId instead of objectId. 

In [14]:
forced_src['ccdVisitId'] 

forcedSourceId
619654830914273628    1154197065
619654830914273629    1154197065
619654830914273630    1154197065
619654830914273631    1154197065
619654830914273632    1154197065
                         ...    
655877493630176923    1221667032
655877493630176924    1221667032
655877493630176925    1221667032
655877493630176926    1221667032
655877493630176927    1221667032
Name: ccdVisitId, Length: 16922251, dtype: int64

In [23]:
# ccdvisit[forced_src['ccdVisitId'].values]
# ccdvisit.loc(forced_src['ccdVisitId'].values)
sourceTable_withTime = forced_src.assign(expMidptMJD = ccdvisit.loc[forced_src['ccdVisitId'].values].values)

In [24]:
sourceTable_withTime.columns

Index(['objectId', 'coord_ra', 'coord_dec', 'psfFlux', 'psfFluxErr',
       'ccdVisitId', 'band', 'expMidptMJD'],
      dtype='object')

In [15]:
len(np.unique(forced_src['objectId']))

39773

Try pip installing TAPE, then run lightkurve tools

TAPE takes a src table (and, optionally, object table) as input

In [25]:
# !pip install lf-tape

In [26]:
from tape import Ensemble, ColumnMapper

In [27]:
ens = Ensemble()

In [28]:
column_mapper = ColumnMapper(id_col='objectId', time_col='expMidptMJD', flux_col='psfFlux',
                             err_col='psfFluxErr', band_col='band')
ens.from_pandas(source_frame=sourceTable_withTime, object_frame=objtable, npartitions=4,
                column_mapper=column_mapper)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/dask/dataframe/core.py:5446: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/home/jcarlin/.local/lib/python3.11/site-packages/tape/ensemble.py:1963: UserWarning: Divisions are not known, syncing using a non-lazy method.
  warnings.warn("Divisions are not known, syncing using a non-lazy method.")
/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 1.09 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/jcarlin/.local/lib/python3.11/site-packages/tape/ensemble.py:1985: UserWarning: Divisions are not known, syncing using a non-lazy method.
  warnings.warn("Divisions are not known, syncing using a non-lazy method.")
/home/jcarlin/.local/lib/python3.11/site-packages

In [32]:
import light_curve as licu

In [30]:
otsu_features = ens.batch(licu.OtsuSplit(), band_to_calc='g', use_map=False)

Using generated label, result_1, for a batch result.


In [31]:
otsu_features.compute()

,otsu_mean_diff,otsu_std_lower,otsu_std_upper,otsu_lower_to_all_ratio
objectId,,,,
1649927148640665601,395.718303,137.993112,149.463054,0.384615
1649927148640665602,148.898756,44.208550,57.413679,0.600000
1649927148640665603,133.357454,40.979675,54.307713,0.550000
1649927148640665604,211.564265,76.367656,57.299600,0.146341
1649927148640665605,192.611612,63.681891,61.279205,0.780488
...,...,...,...,...
1649927148640719739,147.778925,47.990956,67.104367,0.560976
1649927148640719740,127.535266,47.758302,55.368316,0.487179
1649927148640719741,128.582067,51.083263,47.367398,0.560976


In [28]:
ens.source.head(100)

,coord_ra,coord_dec,psfFlux,psfFluxErr,ccdVisitId,band_caller,visitId,physical_filter,band_other,ra,...,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec
objectId,,,,,,,,,,,,,,,,,,,,,
1649927148640665603,56.766040,-37.223510,-72.211329,71.863728,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665604,56.761784,-37.223569,144.014276,73.247223,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665605,56.755506,-37.223464,752.371726,76.601724,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665607,56.735204,-37.222980,81.080547,72.858196,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665608,56.727461,-37.223597,-79.685723,72.273233,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649927148640665735,56.597783,-37.221377,4.327692,73.411160,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665736,56.694428,-37.220976,369.254412,74.895559,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277
1649927148640665737,56.663270,-37.221298,222.916598,74.417111,1154197065,g,1154197,g_sim_1.4,g,56.637288,...,4071,3999,56.568334,-37.298837,56.452562,-37.09292,56.706002,-37.001598,56.822406,-37.207277


In [37]:
licu.Periodogram?

Init signature:
licu.Periodogram(
    peaks=Ellipsis,
    resolution=Ellipsis,
    max_freq_factor=Ellipsis,
    nyquist='average',
    fast=True,
    features=None,
    transform=None,
)
Docstring:     
Peaks of Lomb–Scargle periodogram and periodogram as a meta-feature

Periodogram $P(\omega)$ is an estimate of spectral density of unevenly time series. `peaks` argument
corresponds to a number of the most significant spectral density peaks to return. For each peak its
period and "signal to noise" ratio is returned:

$$
\mathrm{signal~to~noise~of~peak} \equiv \frac{P(\omega_\mathrm{peak}) - \langle P(\omega) \rangle}{\sigma\_{P(\omega)}}.
$$

[Periodogram] can accept other features for feature extraction from periodogram as it was time
series without observation errors (unity weights are used if required). You can even pass one
[Periodogram] to another one if you are crazy enough

- Depends on: **time**, **magnitude**
- Minimum number of observations: as required by sub-features, but a